In [1]:
demofolder = '../../mid/mid-demo/'
spikefile = '11101_1.isk'
moviefile = 'walk1_ieee_128x128x16384.raw'

In [2]:
with open(demofolder+spikefile, 'r') as f:
    nframes = 0
    nspikes = 0
    for line in f:
        nframes += 1
        try:
            nspikes += int(line.rstrip())
        except ValueError:
            print("Can't read this line as an int. Guessing zero.")
print(nframes)
print(nspikes)

Can't read this line as an int.
Can't read this line as an int.
16384
20910


In [4]:
with open(demofolder+moviefile,'rb') as f:
    print(type(f))
    count = 0
    bigcount = 0
    for thing in f:
        print(len(thing))
        count += 1
        bigcount += len(thing)
print(bigcount)
print(count)

<class '_io.BufferedReader'>
123532465
604940
4893040
16025108
82979045
31964868
894
3306
48
8431742
268435456
10


In [6]:
print(thing[1])

57


In [1]:
import matplotlib.pyplot as plt
import numpy as np
plt.ioff()

In [26]:
(128*128*16384) == bigcount

True

In [47]:
with open(demofolder+moviefile,'rb') as f:
    frame = f.read(128*128)
flframe = np.zeros(len(frame))
for pix in range(len(frame)):
    flframe[pix] = float(frame[pix])
plt.imshow(flframe.reshape((128,128)),cmap='gray')

In [ ]:
with open(demofolder+moviefile,'rb') as f:
    while True:
        frame = f.read(128*128)
        if not frame:
            break
        flframe = np.zeros(len(frame))
        for pix in range(len(frame)):
            flframe[pix] = float(frame[pix])
        plt.imshow(flframe.reshape((128,128)),cmap='gray')
        plt.show()

In [12]:
plt.show()

In [18]:
def downsample_frame(frame, factor):
    """Downsample by blocking the image. Only works for integer downsampling rates."""
    if int(factor) != factor:
        raise ValueError("Downsampling rate must be an integer.")
    winit, hinit = frame.shape
    wf=int(winit/factor)
    hf=int(hinit/factor)
    out = np.zeros((wf,hf))
    for x in range(wf):
        for y in range(hf):
            out[x,y] = np.mean(frame[x*factor:(x+1)*factor,y*factor:(y+1)*factor])
    return out

In [19]:
class MovieHandler:
    def __init__(self, moviefile=None, spikefile=None, framesinstim=3, downfactor = 4):
        self.framesinstim = framesinstim
        demofolder = '../../mid/mid-demo/'
        self.downfactor = downfactor
        if moviefile is None:
            moviefile = demofolder +'walk1_ieee_128x128x16384.raw'
        if spikefile is None:
            spikefile = demofolder +'11101_1.isk'
        self.load_movie(moviefile)  
        self.load_spikes(spikefile)
        if len(self.spiketrain) != len(self.movie):
            print('Warning. Spike train length '+str(len(self.spiketrain))+ ' is not equal to movie length '+str(len(self.movie))+'.')        
        
    def get_nstim(self):
        return len(self.spiketrain) - (self.framesinstim-1)
    
    def load_spikes(self, file):
        spiketrain = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    spiketrain.append(int(line.rstrip()))
                except ValueError:
                    print("Can't read this line of spike file as an int. Guessing zero.")
                    spiketrain.append(0)
        self.spiketrain = spiketrain
    
    def load_movie(self, file, frameshape = (128,128)):
        framesize = np.prod(frameshape)
        self.movie = []
        with open(file,'rb') as f:
            while True:
                frame = f.read(framesize)
                if not frame:
                    break
                flframe = np.zeros(len(frame))
                for pix in range(len(frame)):
                    flframe[pix] = float(frame[pix])
                flframe = downsample_frame(flframe.reshape(frameshape),self.downfactor)
                self.movie.append(flframe.flatten())     
        self.frameshape = tuple([int(length/self.downfactor) for length in frameshape])

    def play(self, rate=30):
        try:
            self.movie
        except NameError:
            print("No movie loaded.")
            return
        img = None
        for frame in self.movie:
            fr = frame.reshape(self.frameshape)
            if img is None:
                img = plt.imshow(fr,cmap='gray')
            else:
                img.set_data(fr)
            plt.pause(1./rate)
            plt.draw()
            
    def generator(self):
        lags = self.framesinstim
        try: 
            for t in range(len(self.spiketrain)):
                if t >= lags-1:
                    stim = np.zeros((lags,np.prod(self.frameshape)))
                    for l in range(lags):
                         stim[l,:] = self.movie[t-l].flatten()
                    yield stim.flatten(), self.spiketrain[t]
        #try:
        #    for frame, sp in zip(self.movie, self.spiketrain):
        #            yield frame.flatten(), sp
        except NameError:
            print("Looks like you haven't loaded a movie and spiketrain.")
            raise        
            
    def show_strf(self, strf):
        images = strf.reshape((self.framesinstim,)+self.frameshape)
        plt.figure()
        sqrtsubplots = int(np.ceil((np.sqrt(self.framesinstim))))
        for l in range(self.framesinstim):
            plt.subplot(sqrtsubplots, sqrtsubplots, l+1)
            plt.imshow(images[l], cmap='gray', interpolation='nearest')
            plt.colorbar()

In [20]:
mh = MovieHandler()

Can't read this line of spike file as an int. Guessing zero.
Can't read this line of spike file as an int. Guessing zero.


In [9]:
mh.play()

C:\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


KeyboardInterrupt: 

In [2]:
import mid

Python 3_5 is not supported.
Can't make spectrograms because Matlab engine not compatible. Try using Python 3.4.


In [11]:
from imp import reload
reload(mid)

<module 'mid' from 'C:\\Users\\Eric\\Documents\\Berkeley\\Research\\Neuroscience\\DeWeese_Lab_analysis\\mymid\\MID\\mid.py'>

In [22]:
anmid = mid.MID(handler=mh)

Specialized initialization failed. Falling back on first stimulus.


In [39]:
res = anmid.optimize('GA', rate=1e-4,maxiter=100)

Info             Gradient norm
0.644174259362  5.85793208946
0.644146930621  5.85479589933
0.644441098891  5.84688544667
0.644679063  5.84813821417
0.644500196091  5.83644223234
0.644529629331  5.83306223581
0.644343635146  5.82840801544
0.644805631209  5.83644166373
0.645026340491  5.84722909784
0.644763993589  5.82797676494
0.645242372362  5.84074541769
0.644846082974  5.81160225378
0.645502832802  5.85673062338
0.645206715995  5.83848536091
0.645648391992  5.85874941012
0.645610100775  5.85419850518
0.645337898515  5.82332963789
0.646151412088  5.83982099651
0.645620316393  5.82991029024
0.646568857457  5.85177734683
0.646142141474  5.82965258383
0.647243286943  5.85656891911
0.647540247893  5.8602872292
0.647634571004  5.86036993133
0.646808027907  5.8306724738
0.647879929938  5.8577980335
0.647910764268  5.85208500811
0.647751795076  5.842114405
0.64742906052  5.82443552425
0.648048732425  5.83980094534
0.647993480596  5.83157669404
0.648053166782  5.83213321594
0.648310065359  5.

In [40]:
mh.show_strf(anmid.v)

In [41]:
plt.show()

In [53]:
anmid.v =prettygood

In [3]:
class TestHandler:
    def __init__(self, datasize=5, ndata=1e5):
        self.nstim = int(ndata)
        self.data = np.random.randn(self.nstim,5)
        self.data = self.data/np.sqrt(np.sum(self.data**2,1))[:,np.newaxis]
        self.truerf = np.zeros((datasize))
        self.truerf[0] = 1
        self.generate_spiketrain()
        
    def get_nstim(self):
        return self.nstim
        
    def generate_spiketrain(self):
        ndata = self.nstim
        self.spiketrain = np.zeros((ndata))
        for point in range(ndata):
            rate = self.truerf.dot(self.data[point])
            rate = rate if rate>0 else 0
            self.spiketrain[point] = np.random.binomial(1,rate)
                           
    def generator(self):
        for stim, spike in zip(self.data, self.spiketrain):
            yield stim, spike

In [7]:
th = TestHandler(ndata=1e6)
testmid = mid.MID(handler=th)

Specialized initialization failed. Falling back on first stimulus.


In [13]:
testresults = testmid.optimize('GA',rate=1,maxiter=100)

Info             Gradient norm
0.107123777651  0.05396179066
0.142267658332  0.0628167544264
0.190021646439  0.0721883197348
0.250955804573  0.0821195105054
0.329965688857  0.0913879237723
0.424564603342  0.09866080305
0.535754031468  0.105397030906
0.657868699475  0.108928219932
0.786735071049  0.10983075188
0.91427404664  0.106346395109
1.03038305572  0.0984944789149
1.12789277592  0.0884025443502
1.20105367349  0.0741574783021
1.25511486608  0.0580739101234
1.28374528411  0.0458175285926
1.30036377123  0.0332544346966
1.30894012626  0.0233604804109
1.31313760413  0.00628217816262
1.3123066151  0.00495882479502
1.31254606497  0.00410744765102
1.31299940147  0.00363294017288
1.31269420411  0.00350970443528
1.31279881767  0.00338602593697
1.31305368996  0.00332638083555
1.31296344803  0.00330953619209
1.31302301611  0.00329457814082
1.31290104386  0.003301457799
1.31297772781  0.00329038238481
1.31294674628  0.00329234803067
1.31292451937  0.00329453899985
1.31290184626  0.003295745731

In [14]:
testmid.v

array([-1.35483114, -0.00220391, -0.00385917, -0.00945543, -0.00187351])

In [18]:
th.truerf

array([ 1.,  0.,  0.,  0.,  0.])

In [10]:
info, grad = testmid.info_grad(testmid.v,neg=False)

In [11]:
grad

array([ -3.19422677e-03,   4.53008849e-04,  -8.95012687e-05,
         5.58564394e-05,  -5.67877378e-05])

In [15]:
grad.dot(testmid.v)

NameError: name 'grad' is not defined

In [15]:
v = np.array([.4,.2,.2,.2,.2])
grad = np.zeros(5)
epsilon = .01
info, supposedgrad = testmid.info_grad(v, neg=False)
for direc in range(5):
    delta = np.zeros(5)
    delta[direc] = epsilon
    newinfo, _ =  testmid.info_grad(v+delta, neg=False)
    grad[direc] = (newinfo-info)/epsilon
print(supposedgrad)
print(grad)

[ 0.14781714 -0.06893542 -0.06967218 -0.06996106 -0.06930162]
[ 0.20287896 -0.09543404 -0.10106793 -0.09942686 -0.11212613]


In [16]:
info, pv, pvt = testmid.info_and_dists()
abscissa = np.arange(15)
plt.plot(abscissa, pv, 'b.', abscissa, pvt, 'r.', abscissa, pvt/pv, 'g.')
plt.show()

In [22]:
np.log2(th.get_nstim()/th.spiketrain.sum())

2.4119016054862952

In [17]:
alltheinfo = testmid.info(th.truerf, neg=False)
print(alltheinfo)

1.31991421587


In [27]:
expbias = (th.get_nstim()/th.spiketrain.sum())/(2*np.log(2))
print(expbias)

3.83883347406


In [5]:
withbfgs = testmid.optimize('BFGS')

         Current function value: -1.320318
         Iterations: 5
         Function evaluations: 56
         Gradient evaluations: 44
Desired error not necessarily achieved due to precision loss.


In [6]:
withbfgs.x

array([ 2.52087593,  0.0077204 , -0.02900514, -0.00861167, -0.00476075])

In [8]:
import time 
init = time.clock()
info = testmid.info()
justinfotime = time.clock() - init
info, grad = testmid.info_grad(testmid.v)
bothtime = time.clock() - justinfotime - init
print(justinfotime)
print(bothtime)

5.140145385436469
8.321890010674267
